<a href="https://colab.research.google.com/github/Filarh/Hello-World/blob/master/DeFooocusfinal_fixed_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🎨 DeFooocus Setup
# @markdown ### Configuración básica

# @markdown ### Selección de versión
version = "Fooocus"  # @param ["DeFooocus", "Fooocus"]

activar_drive = True  # @param {type:"boolean"}
modificar_modelo = True  # @param {type:"boolean"}

# @markdown ### Configuración del modelo
# Se utiliza un parámetro desplegable para elegir el preset deseado.
modelo_seleccion = "SDVN8-ArtXL (Arte y vectores)"  # @param ["ZavyChromaXL_v10 (realismo, fotos)", "SDVN8-ArtXL (Arte y vectores)", "Custom"]
custom_link = ""  # @param {type:"string"}  # Se utiliza solo si se elige "Custom"

aspect_ratio = "896*1216"  # @param {type:"string"}
formato = "safetensors"  # @param {type:"string"}

# @markdown ### Configuración de tokens (opcional)
tokenCIVITAI = ""  # @param {type:"string"}
tokenHUGGINGFACE = ""  # @param {type:"string"}

# @markdown ### Configuración de la interfaz
theme = "dark" # @param ["dark", "light"]
preset = "default"  # @param ["default", "anime", "dpo", "hypersd", "lcm", "lightning", "playground_v2.5", "realistic", "sai", "sd1.5", "spo", "turbo"]
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16"  # @param {type:"string"}

import re

# Determinar el enlace y nombre del modelo en función de la opción seleccionada
if modelo_seleccion == "Custom":
    if custom_link.strip() == "":
        raise ValueError("Para el modelo Custom, por favor proporciona un enlace en 'custom_link'.")
    else:
        link_archivo = custom_link.strip()
        is_custom = True
        nombre_archivo = ""  # Se obtendrá luego mediante aria2c
else:
    is_custom = False
    if modelo_seleccion == "ZavyChromaXL_v10 (realismo, fotos)":
        link_archivo = "https://civitai.com/api/download/models/916744?type=Model&format=SafeTensor&size=full&fp=fp16"
    elif modelo_seleccion == "SDVN8-ArtXL (Arte y vectores)":
        link_archivo = "https://civitai.com/api/download/models/212479?type=Model&format=SafeTensor&size=full&fp=fp16"
    # Se elimina cualquier contenido entre paréntesis y espacios para el nombre del preset (nombre interno para el archivo)
    nombre_archivo = re.sub(r'\(.*?\)', '', modelo_seleccion).replace(" ", "")

# Check if 'Args' is not in the local scope
if 'Args' not in locals():
    # Install dependencies if necessary
    !apt install -y aria2
    !pip install pygit2==1.12.2
    %cd /content

    # Clone the selected version
    if version == "DeFooocus":
        !git clone https://github.com/ehristoforu/DeFooocus.git
        %cd /content/DeFooocus
    else:
        !git clone https://github.com/lllyasviel/Fooocus.git
        %cd /content/Fooocus

    Args = ""

# Import necessary libraries
import os
import json
from google.colab import drive

# Fix: Create the 'cached_download' folder in the 'huggingface_hub' path if it doesn't exist
import sys
huggingface_hub_path = '/usr/local/lib/python3.10/dist-packages/huggingface_hub'
cached_download_path = os.path.join(huggingface_hub_path, "cached_download")
if not os.path.exists(cached_download_path):
    os.makedirs(cached_download_path)

# Function to print in color (Colab compatible)
def print_colored(text, color):
    color_codes = {
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        "reset": "\033[0m"
    }
    print(f"{color_codes.get(color, color_codes['reset'])}{text}{color_codes['reset']}")

# Function to modify JSON with provided parameters
def modificar_json(nombre_archivo: str, link_archivo: str, aspect_ratio: str, formato: str):
    carpeta = '/content/DeFooocus/presets' if version == "DeFooocus" else '/content/Fooocus/presets'
    archivos = os.listdir(carpeta)

    json_encontrado = None
    if 'default.json' in archivos:
        json_encontrado = 'default.json'
        print_colored("Using default.json as a reference", "green")
    else:
        json_encontrado = next((archivo for archivo in archivos if archivo.endswith('.json')), None)
        print_colored("No default.json found, using a random JSON file.", "yellow")

    if json_encontrado:
        with open(os.path.join(carpeta, json_encontrado)) as file:
            data = json.load(file)

        archivo_modelo = f"{nombre_archivo}.{formato}"
        data['default_model'] = archivo_modelo
        data['default_aspect_ratio'] = aspect_ratio
        data['default_refiner'] = ""

        data['checkpoint_downloads'] = {archivo_modelo: link_archivo}

        global Args
        Args = f"--preset {nombre_archivo}"

        nuevo_nombre = f"{nombre_archivo}.json"
        with open(os.path.join(carpeta, nuevo_nombre), 'w') as file:
            json.dump(data, file, separators=(',', ':'))

        return f"Created {nuevo_nombre} with the provided parameters and structure!"
    else:
        return "No JSON files found in the specified folder."

# Function to download the model using aria2 if it does not exist.
# Se añade el parámetro is_custom para saber si se debe dejar que aria2c asigne el nombre automáticamente.
def download_model_aria2(url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE, is_custom=False):
    try:
        model_dir = "/content/DeFooocus/models/checkpoints" if version == "DeFooocus" else "/content/Fooocus/models/checkpoints"
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        if not is_custom:
            archivo_modelo = f"{nombre_archivo}.{formato}"
            destino = os.path.join(model_dir, archivo_modelo)

            if os.path.exists(destino):
                print_colored(f"File {archivo_modelo} already exists. No download required.", "cyan")
                return destino, archivo_modelo

            # Append token if needed
            if "huggingface" in url and tokenHUGGINGFACE:
                url += f"&token={tokenHUGGINGFACE}"
            elif "civitai" in url and tokenCIVITAI:
                url += f"&token={tokenCIVITAI}"

            download_command = f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}" -o "{archivo_modelo}"'
            print_colored(f"Downloading from URL: {download_command}", "blue")
            os.system(download_command)

            if os.path.exists(destino):
                print_colored(f"Downloaded file at: {destino}", "green")
                return destino, archivo_modelo
            else:
                print_colored("Failed to download file. ¿si agregaste la clave API?", "red")
                return None, None
        else:
            # Para descarga custom: no se especifica nombre (flag -o) y se obtiene el nombre descargado
            existing_files = set(os.listdir(model_dir))

            if "huggingface" in url and tokenHUGGINGFACE:
                url += f"&token={tokenHUGGINGFACE}"
            elif "civitai" in url and tokenCIVITAI:
                url += f"&token={tokenCIVITAI}"

            download_command = f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}"'
            print_colored(f"Downloading (custom) from URL: {download_command}", "blue")
            os.system(download_command)

            new_files = set(os.listdir(model_dir)) - existing_files
            new_files = [f for f in new_files if f.endswith(f".{formato}")]
            if len(new_files) == 0:
                print_colored("Failed to download file. ¿si agregaste la clave API?", "red")
                return None, None
            elif len(new_files) == 1:
                archivo_modelo = new_files[0]
            else:
                # Si se descargaron varios, se elige el más reciente
                archivo_modelo = sorted(new_files, key=lambda f: os.path.getmtime(os.path.join(model_dir, f)), reverse=True)[0]
            destino = os.path.join(model_dir, archivo_modelo)
            print_colored(f"Downloaded file at: {destino}", "green")
            return destino, archivo_modelo

    except Exception as e:
        print_colored(f"Download error: {e}", "red")
        return None, None

# Activating Google Drive if necessary
if activar_drive:
    drive.mount('/content/drive')
    nuevo_dir_lora = '/content/drive/MyDrive/MyLoras'
    nuevo_dir_salida = '/content/drive/MyDrive/Fooocus_output'

    launch_py_path = '/content/DeFooocus/launch.py' if version == "DeFooocus" else '/content/Fooocus/launch.py'

    if os.path.exists(launch_py_path):
        with open(launch_py_path, 'r') as file:
            launch_py_content = file.read()

        codigo_a_insertar = f'''
import os
import json

nuevo_dir_lora = '{nuevo_dir_lora}'
nuevo_dir_salida = '{nuevo_dir_salida}'

config_path = os.path.abspath("./config.txt")
config_dict = {{}}

if os.path.exists(config_path):
    with open(config_path, "r", encoding="utf-8") as json_file:
        config_dict = json.load(json_file)

config_dict['path_loras'] = os.path.abspath(nuevo_dir_lora)
config_dict['path_outputs'] = os.path.abspath(nuevo_dir_salida)

with open(config_path, "w", encoding="utf-8") as json_file:
    json.dump(config_dict, json_file, indent=4)
'''

        if codigo_a_insertar not in launch_py_content:
            with open(launch_py_path, 'w') as file:
                file.write(codigo_a_insertar + launch_py_content)
        else:
            print_colored("Code already inserted in launch.py", "cyan")
    else:
        print_colored("launch.py file not found.", "red")

# Se descarga primero el modelo; en el caso custom se obtendrá el nombre descargado.
model_url = link_archivo
model_sd, downloaded_filename = download_model_aria2(model_url, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE, is_custom=is_custom)
if model_sd:
    print_colored(f"Model downloaded: {model_sd}", "green")
    if is_custom:
        # Se utiliza el nombre obtenido del archivo, quitándole la extensión y eliminando espacios
        nombre_archivo = os.path.splitext(downloaded_filename)[0].replace(" ", "")
else:
    print_colored("Model download failed.", "red")

# Si se requiere modificar el modelo (JSON), se actualiza con el nombre final y el enlace original.
if modificar_modelo:
    resultado = modificar_json(nombre_archivo, link_archivo, aspect_ratio, formato)
    print(resultado)

print_colored("[DeFooocus] Preparing...", "magenta")

if preset != "default":
    args = f"{advenced_args} --theme {theme} --preset {preset}"
else:
    args = f"{advenced_args} --theme {theme}"

args = f"{args} {Args}"

print_colored("[DeFooocus] Starting...", "magenta")
!python entry_with_update.py {args}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
  Using cached pygit2-1.12.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
Using cached pygit2-1.12.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
  Attempting uninstall: pygit2
    Found existing installation: pygit2 1.15.1
    Uninstalling pygit2-1.15.1:
      Successfully uninstalled pygit2-1.15.1
/content
fatal: destination path 'Fooocus' already exists and is not an empty directory.
/content/Fooocus
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Code already inserted in launch.py
Downloaded file at: /content/Fooocus/models/checkpoints/SDVN8-ArtXL.safetensors
Model downloaded: /content/Fooocus/models/checkpoints/SDVN8-ArtXL.safetensors
Using d